# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [1]:
!pip install -q -U langchain openai

In [2]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API Key:  ········


### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [3]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [4]:
!pip install -q -U wikipedia

In [13]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)", 
    load_max_docs=1,
    doc_content_chars_max=1_000_000
    ).load()

barbie_csv_docs = CSVLoader(
    file_path='./barbie_data/barbie.csv',
    source_column="Review"
    ).load()

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 0,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n==", "\n", " "] # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n", " "] # keep paragraphs, then sentences
)

chunked_barbie_wikipedia_docs = wikipedia_text_splitter.split_documents(barbie_wikipedia_docs)
chunked_barbie_csv_docs = csv_text_splitter.split_documents(barbie_csv_docs)

In [27]:
chunked_barbie_wikipedia_docs[2]

Document(page_content='A live-action Barbie film was announced in September 2009 by Universal Pictures with Laurence Mark producing. Development began in April 2014, when Sony Pictures acquired the film rights. Following multiple writer and director changes and the casting of Amy Schumer and later Anne Hathaway as Barbie, the rights were transferred to Warner Bros. Pictures in October 2018. Robbie was cast in 2019, after Gal Gadot turned down the role due to scheduling conflicts, and Gerwig was announced as director and', metadata={'title': 'Barbie (film)', 'summary': 'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast,

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [18]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

In [28]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./cache/")

core_embeddings_model = OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace=core_embeddings_model.model
)

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [29]:
barbie_csv_faiss_retriever = FAISS.from_documents(chunked_barbie_csv_docs, embedder).as_retriever() ### YOUR CODE HERE


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [32]:
len(chunked_barbie_csv_docs)

180

In [33]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_barbie_wikipedia_docs ### YOUR CODE HERE
)
barbie_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
barbie_wikipedia_faiss_store = FAISS.from_documents(
    chunked_barbie_wikipedia_docs, ### YOUR CODE HERE
    embedder ### YOUR CODE HERE
)
barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever], ### YOUR CODE HERE
    weights=[0.2, 0.8] ### YOUR CODE HERE # should sum to 1
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [39]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    retriever=barbie_ensemble_retriever, ### YOUR CODE HERE 
    name="Wikipedia", ### YOUR CODE HERE
    description="Useful for when you need to answer questions about plot, cast, production, music, history of the Barbie movie" ### YOUR CODE HERE
)

barbie_csv_retrieval_tool = create_retriever_tool(
    retriever=barbie_csv_faiss_retriever, ### YOUR CODE HERE
    name="Reviews", ### YOUR CODE HERE
    description="Useful for when you need to answer questions about public reviews of the Barbie movie"### YOUR CODE HERE
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool] ### YOUR CODE HERE

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [40]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(
    llm=llm, 
    tools=barbie_retriever_tools,
    verbose=True
) # YOUR CODE HERE

In [37]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `Barbie`


[Document(page_content="for details and her ability to create a dreamlike world. Barbieland is a feminist utopia in which Barbies serve as presidents, Supreme Court members, doctors, physicists, Nobel prize winners, and cancer cures. The dolls are not structurally different from our world, but they are represented by political fantasy and decade-dependent what-ifs. Barbie has gone to space, could vote, and own property before many human women, and her appearance has changed to mirror society's beauty politics. The representational doll has become an extension of political fantasy, reflecting the beauty politics of society. Gerwig features a cast of Barbies, including Issa Rae, Emma Mackey, Alexandra Shipp, and Hari Nef. Stereotypical Barbie (Margot Robbie) is the protagonist, a blonde-haired, blue-eyed Barbie created by Ruth Handler. Her Ken counterpart, Ryan Gosling, is portrayed with humor and heart. They ar

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Reviews', 'arguments': '{\n  "__arg1": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content="for details and her ability to create a dreamlike world. Barbieland is a feminist utopia in which Barbies serve as presidents, Supreme Court members, doctors, physicists, Nobel prize winners, and cancer cures. The dolls are not structurally different from our world, but they are represented by political fantasy and decade-dependent what-ifs. Barbie has gone to space, could vote, and own property before many human women, and her appearance has changed to mirro

In [41]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `Barbie (film series)`


[Document(page_content='Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu, Michael', metadata={'title': 'Barbie (film)', 'summary': 'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [HumanMessage(content='What is a very quick summary of the plot of the Barbie movie?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Wikipedia', 'arguments': '{\n  "__arg1": "Barbie (film series)"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu, Michael\', metadata={\'title\': \'Barbie (film)\', \'summary\': \'Barbie 

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [44]:
oppenheimer_wikipedia_docs = WikipediaLoader(
    query="Oppenheimer (film)", 
    load_max_docs=1, 
    doc_content_chars_max=1_000_000
).load() ### YOUR CODE HERE

oppenheimer_csv_docs = CSVLoader(
    file_path="./oppenheimer_data/oppenheimer.csv", 
    source_column="Review_Url"
).load() ### YOUR CODE HERE

In [45]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.transform_documents(oppenheimer_wikipedia_docs) ### YOUR CODE HERE
chunked_opp_csv_docs = csv_text_splitter.transform_documents(oppenheimer_csv_docs) ### YOUR CODE HERE

In [46]:
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_csv_docs, embedder).as_retriever() ### YOUR CODE HERE

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_opp_wikipedia_docs ### YOUR CODE HERE
)
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(
    chunked_opp_wikipedia_docs, 
    embedder 
)
opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever], ### YOUR CODE HERE
    weights=[0.2, 0.8] ### YOUR CODE HERE # should sum to 1
)

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [47]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [48]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [49]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content="People generally thought highly of the Oppenheimer movie. Critics praised the screenplay, performances of the cast (especially Cillian Murphy and Robert Downey Jr.), and the visuals. It was frequently cited as one of the best films of 2023. On Rotten Tomatoes, 93% of 450 critics' reviews were positive, with an average rating of 8.6/10. Audience reviews also expressed positive opinions, with viewers finding the movie compelling, engaging, and simple to understand. The actors were praised for their phenomenal performances, particularly Cillian Murphy. Some viewers appreciated the film's cinematography and its ability to spark an interest in history. However, there were some criticisms regarding the historical accuracy and the absence of emotional depth in the portrayal of J. Robert Oppenheimer.", additional_kwargs={}, example=False)

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [55]:
from langchain.agents import Tool

def query_oppenheimer(input):
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name="OppenheimerInfo",
        func=query_oppenheimer, ### YOUR CODE HERE,
        description="Useful for when you need to answer questions about public reviews, plot, cast, production, history of the Oppenheimer movie. Input should be a fully formed question." ### YOUR CODE HERE
    ),
    Tool(
        name ="BarbieInfo",  ### YOUR CODE HERE
        func=barbie_retriever_agent_executor.invoke, ### YOUR CODE HERE
        description="Useful for when you need to answer questions about public reviews, plot, cast, production, history of the Barbie movie. Input should be a fully formed question." ### YOUR CODE HERE
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [56]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools=tools, ### YOUR CODE HERE
    prefix=prefix, ### YOUR CODE HERE
    suffix=suffix, ### YOUR CODE HERE
    input_variables=['input', 'agent_scratchpad'] ### YOUR CODE HERE
)

In [57]:
from langchain import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt) ### YOUR CODE HERE

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [58]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain=llm_chain, ### YOUR AGENTS, 
    tools=tools, ### YOUR AGENTS, 
    verbose=True ### YOUR AGENTS
)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=barbenheimer_agent, ### YOUR AGENTS
    tools=tools, ### YOUR AGENTS 
    verbose=True ### YOUR AGENTS
)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [59]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what people liked about the Barbie movie.
Action: BarbieInfo
Action Input: What did people like about the Barbie movie?

> Entering new AgentExecutor chain...
Based on the reviews, here are some things that people liked about the Barbie movie:

1. Visual Spectacle: The movie is praised for its wonderful visual spectacle, with creative physical comedy and a doll-like aesthetic. The use of colorful visuals and doll-like movements of the actors are appreciated.

2. Fun and Entertainment: Many viewers found the movie to be fun and entertaining. The comedy in the film is mentioned as funny, and there are choreographed dances that add to the enjoyment.

3. Empowerment Theme: Some viewers appreciated the theme of empowerment in the movie. It resonated with them and made the film a big hit.

4. Darkly Satirical Look at Gender Structure: The movie is commended for taking a darkly satirical look at gender structure in society. I

{'input': 'What did people like about the Barbie movie?',
 'output': 'Based on the reviews, people liked the Barbie movie for its wonderful visual spectacle, fun and entertainment, empowerment theme, darkly satirical look at gender structure, and the performances of the actors.'}

In [60]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what people liked about the Oppenheimer movie.
Action: OppenheimerInfo
Action Input: "What did people like about the Oppenheimer movie?"
Observation: content='People liked the screenplay, the performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised the film for being compelling, engaging, and simple to understand. The cinematography was also highly praised, with critics describing it as beautifully crafted. Additionally, the film received acclaim for its cast performances, screenplay, and visuals.' additional_kwargs={} example=False
Thought:Based on the information I found, people liked the screenplay, cast performances, visuals, and cinematography of the Oppenheimer movie.
Final Answer: People liked the screenplay, cast performances, visuals, and cinematography of the Oppenheimer movie.

> Finished chain.


'People liked the screenplay, cast performances, visuals, and cinematography of the Oppenheimer movie.'

In [ ]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: